In [1]:
# De ser necesario
!pip install altair vega_datasets

In [ ]:
import pandas as pd
import altair as alt
import os

# Definir la ruta del directorio de resultados en Google Drive
results_dir = "/content/drive/My Drive/TFM/resultados"

# Cargar los datos desde Google Drive
results_df = pd.read_csv(os.path.join(results_dir, "resultados_modelo.csv"))
results_df_all_models = pd.read_csv(os.path.join(results_dir, "resultados_todos_modelos.csv"))
all_predictions_df = pd.read_csv(os.path.join(results_dir, "todas_las_predicciones.csv"))

# Conversión explícita de tipos de datos para evitar warnings
results_df_all_models['RMSE'] = pd.to_numeric(results_df_all_models['RMSE'], errors='coerce')
results_df_all_models['R^2'] = pd.to_numeric(results_df_all_models['R^2'], errors='coerce')
results_df_all_models['Modelo'] = results_df_all_models['Modelo'].astype(str)

# Identificar el mejor modelo
best_model_name = results_df_all_models.loc[results_df_all_models['RMSE'].idxmin(), 'Modelo']

### Creación del Dashboard

# Crear un selector interactivo para los modelos que afecte a todos los gráficos
model_selector = alt.selection_multi(fields=['Modelo'], bind='legend')

# Gráfico de RMSE con selector
rmse_chart = alt.Chart(results_df_all_models).mark_bar().encode(
    x=alt.X('RMSE:Q', title='RMSE'),
    y=alt.Y('Modelo:N', sort='-x', title='Modelo'),
    tooltip=['Modelo', 'RMSE'],
    color=alt.condition(model_selector, 'Modelo:N', alt.value('lightgray'))
).add_selection(
    model_selector
).properties(
    title='Resultados de Todos los Modelos (RMSE)'
)

# Gráfico de R² con selector
r2_chart = alt.Chart(results_df_all_models).mark_bar().encode(
    x=alt.X('R^2:Q', title='R²', scale=alt.Scale(domain=[-0.3, 1])),  # Ajustar dominio de R²
    y=alt.Y('Modelo:N', sort='-x', title='Modelo'),
    tooltip=['Modelo', 'R^2'],
    color=alt.condition(model_selector, 'Modelo:N', alt.value('lightgray'))
).add_selection(
    model_selector
).properties(
    title='Resultados de Todos los Modelos (R²)'
)

# Tabla que muestra los valores de RMSE y R² sin duplicar el nombre del modelo
rmse_r2_table = alt.Chart(results_df_all_models).mark_text(align='left', baseline='middle').encode(
    y=alt.Y('Modelo:N', title='Modelo', axis=alt.Axis(labelAngle=0)),
    text=alt.Text('RMSE:Q', format='.4f'),
    color=alt.value('black')
)

r2_values = alt.Chart(results_df_all_models).mark_text(align='left', baseline='middle').encode(
    y=alt.Y('Modelo:N', title=None),
    text=alt.Text('R^2:Q', format='.4f'),
    color=alt.value('black')
)

# Unir RMSE y R² en una sola tabla con dos columnas, centrado
rmse_r2_combined = alt.layer(
    rmse_r2_table.encode(x=alt.value(100), text='RMSE:Q'),  # Ajuste para mostrar RMSE a la izquierda
    r2_values.encode(x=alt.value(200), text='R^2:Q')  # Ajuste para mostrar R² a la derecha
).properties(
    title='Tabla de RMSE y R² para Todos los Modelos',
    width=300
)

# Gráfico de predicciones de todos los modelos vs. valores reales con selector
all_preds_long = all_predictions_df.melt(id_vars=['Valores Reales'],
                                         var_name='Modelo', value_name='Predicciones')

# Gráfico de todas las predicciones vs valores reales
all_preds_chart = alt.Chart(all_preds_long).mark_circle(size=40, opacity=0.5).encode(
    x=alt.X('Valores Reales:Q', title='Valores Reales', scale=alt.Scale(domain=[40, 90])),
    y=alt.Y('Predicciones:Q', title='Predicciones', scale=alt.Scale(domain=[40, 90])),
    color=alt.condition(model_selector, 'Modelo:N', alt.value('lightgray')),
    tooltip=['Valores Reales', 'Predicciones', 'Modelo']
).add_selection(
    model_selector
).properties(
    title='Comparación de Predicciones de Todos los Modelos vs. Valores Reales'
)

# Añadir la línea de identidad al gráfico de todas las predicciones
identity_line = alt.Chart(results_df).mark_line(color='red', size=2).encode(
    x='Valores Reales',
    y='Valores Reales'
)

# Combinar el gráfico de predicciones con la línea de identidad
all_preds_with_identity = all_preds_chart + identity_line

# Gráfico de Predicciones vs Valores Reales para el Mejor Modelo
pred_vs_real_chart = alt.Chart(results_df).mark_circle(size=60).encode(
    x=alt.X('Valores Reales:Q', title='Valores Reales', scale=alt.Scale(domain=[40, 90])),  # Ajustar el dominio del eje X
    y=alt.Y('Predicciones:Q', title='Predicciones', scale=alt.Scale(domain=[40, 90])),  # Ajustar el dominio del eje Y
    tooltip=['Valores Reales', 'Predicciones']
).properties(
    title=f'Predicciones vs. Valores Reales ({best_model_name}) - Mejor Modelo'
)

# Añadir la línea de identidad al gráfico de predicciones vs valores reales
pred_vs_real_with_identity = pred_vs_real_chart + identity_line

# Combinar los gráficos en un solo Dashboard interactivo, gráficos 3 y 4 lado a lado
dashboard = alt.vconcat(
    alt.hconcat(rmse_chart, r2_chart),
    rmse_r2_combined,
    alt.hconcat(all_preds_with_identity, pred_vs_real_with_identity)  # Gráficos 3 y 4 lado a lado
).resolve_scale(
    color='independent'
)

# Mostrar el Dashboard
dashboard.display()


/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)